In [ ]:
! pip install langchain==0.1.2 openai

In [1]:
# 加载环境
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name='gpt-3.5-turbo-instruct',temperature=0.0)

In [4]:
class Joke(BaseModel):
    setup: str = Field(description='question  to set up a joke')
    punchline: str = Field(description='answer to resolve the joke')

    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError('Badly formed question!')
        return field
    

parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template='Answer the user query.\n{format_instructions}\n{query}\n',
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions()},
)

prompt_and_model = prompt | model
output = prompt_and_model.invoke({'query':'Tell me a joke.'})
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

Connection error caused failure to post https://api.smith.langchain.com/runs  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))"))
Connection error caused failure to patch https://api.smith.langchain.com/runs/3cc2e459-c91e-40f2-a737-843946fbbbad  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/3cc2e459-c91e-40f2-a737-843946fbbbad (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))"))


In [5]:
chain = prompt | model | parser
chain.invoke({'query':'Tell me a joke.'})

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

In [6]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question} "
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | model | json_parser

list(json_chain.stream({'question':'Who invented the microscope'}))

[{},
 {'answer': ''},
 {'answer': 'Ant'},
 {'answer': 'Anton'},
 {'answer': 'Antonie'},
 {'answer': 'Antonie van'},
 {'answer': 'Antonie van Lee'},
 {'answer': 'Antonie van Leeu'},
 {'answer': 'Antonie van Leeuwen'},
 {'answer': 'Antonie van Leeuwenho'},
 {'answer': 'Antonie van Leeuwenhoek'}]

In [7]:
list(chain.stream({"query": "Tell me a joke."}))

[Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')]

## CSV解析器
当您希望返回以逗号分隔的项目列表时，可以使用此输出解析器。

In [8]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template='List five {subject}.\n{format_instructions}',
    input_variables=['subject'],
    partial_variables={'format_instructions':format_instructions},
)

model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser

In [9]:
chain.invoke({'subject':'ice cream flavors'})

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

In [10]:
for s in chain.stream({'subject':'ice cream flavors'}):
    print(s)

['Vanilla']
['Chocolate']
['Strawberry']
['Mint Chocolate Chip']
['Cookies and Cream']


## 日期时间解析器

In [11]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()
template = """Answer ther users question: 

{question}

{format_instructions}"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={'format_instructions':output_parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0155-08-07T04:41:18.252414Z, 1012-10-24T03:12:14.027451Z, 1861-06-16T19:41:35.903114Z\n\nReturn ONLY this string, no other words!"}, template='Answer ther users question: \n\n{question}\n\n{format_instructions}')

In [12]:
chain = prompt | OpenAI() | output_parser

In [14]:
output = chain.invoke({'question':'wher was bitcoin founded?'})

print(output)

2009-01-03 18:15:05


## 枚举解析器

In [15]:
from langchain.output_parsers.enum import EnumOutputParser

from enum import Enum

class Colors(Enum):
    RED = 'red'
    GREEN = 'green'
    BLUE = 'blue'


parser = EnumOutputParser(enum=Colors)

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    """What color eyes does this person have?

> Person: {person}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI() | parser

In [17]:
chain.invoke({'person':'Frank Sinatra'})

<Colors.BLUE: 'blue'>

## JSON解析器

In [18]:
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

In [19]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [20]:
joke_query = 'Tell me a joke'

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template='Answer the user query.\n{format_instructions}\n{query}\n',
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({'query': joke_query})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [21]:
for s in chain.stream({"query": joke_query}):
    print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why don'}
{'setup': "Why don't"}
{'setup': "Why don't scientists"}
{'setup': "Why don't scientists trust"}
{'setup': "Why don't scientists trust atoms"}
{'setup': "Why don't scientists trust atoms?"}
{'setup': "Why don't scientists trust atoms?", 'punchline': ''}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}


In [22]:
joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'joke': "Why don't scientists trust atoms? Because they make up everything!"}

## OpenAI 函数
- JsonOutputFunctionsParser
- PydanticOutputFunctionsParser
- JsonKeyOutputFunctionsParser
- PydanticAttrOutputFunctionsParser

In [24]:
from langchain_community.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description='question to set up a joke')
    punchline: str = Field(description='answer to resolve the joke')

openai_functions = [convert_pydantic_to_openai_function(Joke)]

model = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [('system','You are a helpful assistant'),('user', '{input}')]
)

### JsonOutputFunctionsParser

In [25]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

parser = JsonOutputFunctionsParser()

chain = prompt | model.bind(functions=openai_functions) | parser

chain.invoke({'input':'Tell me a joke'})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [26]:
for s in chain.stream({'input':'Tell me a joke'}):
    print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why don'}
{'setup': "Why don't"}
{'setup': "Why don't scientists"}
{'setup': "Why don't scientists trust"}
{'setup': "Why don't scientists trust atoms"}
{'setup': "Why don't scientists trust atoms?"}
{'setup': "Why don't scientists trust atoms?", 'punchline': ''}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}


### JsonKeyOutputFunctionsParser

In [30]:
from typing import List
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

class Jokes(BaseModel):
    """Jokes to tell user"""
    joke: List[Joke]
    funniness_level: int


parser = JsonKeyOutputFunctionsParser(key_name='joke')

openai_functions = [convert_pydantic_to_openai_function(Jokes)]
chain = prompt | model.bind(functions=openai_functions) | parser

In [31]:
chain.invoke({'input':'Tell me two jokes'})

[{'setup': "Why don't scientists trust atoms?",
  'punchline': 'Because they make up everything!'},
 {'setup': 'Why did the scarecrow win an award?',
  'punchline': 'Because he was outstanding in his field!'}]

In [33]:
for s in chain.stream({"input": "tell me two jokes"}):
    print(s)

[]
[{}]
[{'setup': ''}]
[{'setup': 'Why'}]
[{'setup': 'Why don'}]
[{'setup': "Why don't"}]
[{'setup': "Why don't scientists"}]
[{'setup': "Why don't scientists trust"}]
[{'setup': "Why don't scientists trust atoms"}]
[{'setup': "Why don't scientists trust atoms?"}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': ''}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}, {}]
[{'setup': "Why don't scientists trust atoms?", 'pu

### PydanticOutputFunctionsParser

In [34]:
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser

class Joke(BaseModel):
    """Joke to tell user"""

    setup: str = Field(description='question to set up a joke')
    punchline: str = Field(description='answer to resolve the joke')

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

parser = PydanticOutputFunctionsParser(pydantic_schema=Joke)

openai_functions = [convert_pydantic_to_openai_function(Joke)]

chain = prompt | model.bind(functions=openai_functions) | parser

In [35]:
chain.invoke({"input": "tell me a joke"})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

## OpenAI Tools

## Structured output parser

In [1]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(
        name="source",
        description="source used to answer the user's question, should be a website.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [2]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [3]:
model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser

In [4]:
chain.invoke({"question": "what's the capital of france?"})

{'answer': 'The capital of France is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}

In [5]:
for s in chain.stream({"question": "what's the capital of france?"}):
    print(s)

{'answer': 'The capital of France is Paris.', 'source': 'https://en.wikipedia.org/wiki/Paris'}
